In [ ]:
# COMPLETE - how to get zero for a user/date/hour that has no observations?
   # create cartesian product of user/hour/date
# COMPLETE - exclude all but the last 30-days from current day (drop older records, or use window?) 
# COMPLETE - Change to use mean instead of median
# TODO - Should we just focus on a single requirement?
# TODO - Do something with activity? 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime/

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S')
usecols = ["id", "date", "user", "pc", "activity"]
logon_df = pd.read_csv('../data/r2/logon.csv', parse_dates=['date'], date_parser=dateparse, usecols=usecols)
logon_df = logon_df.set_index('date').sort_index(axis=0)

In [ ]:
logon_df.info(memory_usage='deep')

In [ ]:
logon_df.head()

In [ ]:
# filter the data to a specific date range
logon_df = logon_df['2010-02-20':'2010-03-20']

In [ ]:
# create a field for the hour of the day
logon_df['hour'] = logon_df.index.hour
logon_df.head()

In [ ]:
# group the data by user, hour and resample on a daily basis, counting the number of events per day.
# this results in a df that shows the count of logons/logoffs for each user/hour for each day
# if a user had no events during a given day/hour, then no record exists in this 
# dataframe (which is a problem we will fix)

logon_resampled_df = logon_df.groupby(['user','hour'], sort=False).resample('D').count()[['id']]
logon_resampled_df = logon_resampled_df.rename(columns={'id': 'login_count'})
logon_resampled_df = logon_resampled_df.astype({'login_count': 'int32'})
logon_resampled_df.info(memory_usage='deep')

In [ ]:
# this code creates entries that indicate a user had 0 event entries for a given time interval.
# this is necessary because when using a group by, hours in which the user had no activity do not
# show any results and are consequently not included in the calculations. instead, we want the hour/user/date
# combination to show 0 for dates/hours when the user had 0 events for the purpose of computing statistics

users = logon_resampled_df.index.get_level_values(0).unique()
hours = logon_resampled_df.index.get_level_values(1).unique()
dates = logon_resampled_df.index.get_level_values(2).unique()

# create a cartesian product of users, hours and dates
index = pd.MultiIndex.from_product([users, hours, dates], names = ["user", "hour", "date"])
full_df = pd.DataFrame(index = index)

# set the default value for the login_count to 0 - this will be overwritten by the true
# count that was calculated in logon_resampled_df
full_df['login_count'] = 0

# save some memory
full_df = full_df.astype({'login_count': 'int32'})

full_df.info(memory_usage='deep')

In [ ]:
# merge the two data frames on the index
daily_count_df = pd.merge(left=full_df, right=logon_resampled_df, how='left', right_index=True, left_index=True)

# collapse the two login_count columns into a single column and remove the columns created by the merge
daily_count_df['login_count'] = daily_count_df[["login_count_x", "login_count_y"]].max(axis=1)
daily_count_df = daily_count_df.drop(['login_count_x', 'login_count_y'], axis=1)
daily_count_df = daily_count_df.astype({'login_count': 'int32'})

In [ ]:
daily_count_df.info(memory_usage='deep')

## Start of outlier detection calculations

In [ ]:
# find the average login count for each user, hour combination
daily_count_df['mean_login_count'] = daily_count_df.groupby(['user','hour'], sort=False).login_count.mean()
daily_count_df.head()

In [ ]:
# calculate the absolute deviation
daily_count_df['abs_dev'] = abs(daily_count_df['mean_login_count'] - daily_count_df['login_count'])
daily_count_df.head()

In [ ]:
# calculate the mean absolute deviation
daily_count_df['mean_abs_dev'] = daily_count_df.groupby(['user','hour'], sort=False).abs_dev.mean()
daily_count_df.head()

In [ ]:
# calcuate the lower bound and upper bound
multiplier = 9 
daily_count_df['lower_bound'] = daily_count_df['mean_login_count'] - (daily_count_df['mean_abs_dev'] * multiplier)
daily_count_df['upper_bound'] = daily_count_df['mean_login_count'] + (daily_count_df['mean_abs_dev'] * multiplier)

daily_count_df.head()

In [ ]:
# calculate the outliers
daily_count_df['outlier'] = np.where(daily_count_df['login_count'] < daily_count_df['lower_bound'] , 1, 
                                         np.where(daily_count_df['login_count'] > daily_count_df['upper_bound'], 1, 0))
daily_count_df.head()

In [ ]:
# how many users have outliers? 
len(daily_count_df[daily_count_df['outlier'] == 1].index.get_level_values(0).unique())

In [ ]:
# check to see if the one insider in r2 is listed
'ONS0995' in daily_count_df[daily_count_df['outlier'] == 1].index.get_level_values(0).unique()

In [ ]:
#daily_count_df.query('user == "ONS0995" and date >= "2010/03/06" and date < "2010/03/21" and hour==0').head(250)
daily_count_df.query('user == "ONS0995" and hour==0').head(250)

CMU dataset scenarios

1. User who did not previously use removable drives or work after hours begins logging in after hours, using a removable drive, and uploading data to wikileaks.org. Leaves the organization shortly thereafter.

2. User begins surfing job websites and soliciting employment from a
competitor. Before leaving the company, they use a thumb drive (at
markedly higher rates than their previous activity) to steal data.

3. System administrator becomes disgruntled. Downloads a keylogger and
uses a thumb drive to transfer it to his supervisor's machine. The
next day, he uses the collected keylogs to log in as his supervisor
and send out an alarming mass email, causing panic in the
organization. He leaves the organization immediately.

4. A user logs into another user's machine and searches for
interesting files, emailing to their home email. This behavior occurs
more and more frequently over a 3 month period.

5. A member of a group decimated by layoffs uploads documents to
Dropbox, planning to use them for personal gain.